In [1]:
from __future__ import print_function, division
import pypsa

import numpy as np
from scipy.sparse.csgraph import dijkstra

In [2]:
network = pypsa.Network()

In [3]:
# tes tiga bus
n_buses = 3

for i in range(n_buses):
    network.add("Bus","My bus {}".format(i), v_nom=20.)

In [4]:
network.buses

attribute,v_nom,type,x,y,carrier,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,sub_network
My bus 0,20.0,,0.0,0.0,AC,1.0,0.0,inf,PQ,
My bus 1,20.0,,0.0,0.0,AC,1.0,0.0,inf,PQ,
My bus 2,20.0,,0.0,0.0,AC,1.0,0.0,inf,PQ,


In [5]:
# Tambah tiga line dalam ring
for i in range(n_buses):
    network.add("Line", "My line {}".format(i),
               bus0="My bus {}".format(i),
               bus1="My bus {}".format((i+1)%n_buses),
                x = 0.1,
                r = 0.01
               )

In [6]:
network.lines

attribute,bus0,bus1,type,x,r,g,b,s_nom,s_nom_extendable,s_nom_min,...,terrain_factor,num_parallel,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,s_nom_opt
My line 0,My bus 0,My bus 1,,0.1,0.01,0.0,0.0,0.0,False,0.0,...,1.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0
My line 1,My bus 1,My bus 2,,0.1,0.01,0.0,0.0,0.0,False,0.0,...,1.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0
My line 2,My bus 2,My bus 0,,0.1,0.01,0.0,0.0,0.0,False,0.0,...,1.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0


In [7]:
# Tambah Generator pada bus 0
network.add("Generator","My Gen", bus="My bus 0", p_set=100, control="PQ")

In [8]:
network.generators

attribute,bus,control,type,p_nom,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,p_set,...,start_up_cost,shut_down_cost,min_up_time,min_down_time,initial_status,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,p_nom_opt
My Gen,My bus 0,PQ,,0.0,False,0.0,inf,0.0,1.0,100.0,...,0.0,0.0,0,0,1,NaN,NaN,1.0,1.0,0.0


In [9]:
network.generators.p_set

My Gen    100.0
Name: p_set, dtype: float64

In [10]:
# Tambah beban pada bus 1
network.add("Load", "My Load", bus="My bus 1", p_set=100)

In [11]:
network.loads

attribute,bus,type,p_set,q_set,sign
My Load,My bus 1,,100.0,0.0,-1.0


In [12]:
network.loads.p_set

My Load    100.0
Name: p_set, dtype: float64

In [13]:
network.loads.q_set = 100.

In [14]:
# Analisa aliran daya dengan metode newton-raphson
network.pf()

INFO:pypsa.pf:Slack bus for sub-network 0 is My bus 0
INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['now'], dtype='object')
INFO:pypsa.pf:Newton-Raphson solved in 3 iterations with error of 0.000000 in 0.051418 seconds


{'converged':         0
 now  True, 'error':                 0
 now  4.753531e-10, 'n_iter':      0
 now  3}

In [15]:
network.lines_t.p0

,My line 0,My line 1,My line 2
now,66.897487,-33.333333,-33.391038


In [16]:
network.buses_t.v_ang*180/np.pi

,My bus 0,My bus 1,My bus 2
now,0.0,-0.875939,-0.433813


In [17]:
network.buses_t.v_mag_pu

,My bus 0,My bus 1,My bus 2
now,1.0,0.981199,0.99057
